<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M3_CrewAI_Tutorial_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comprehensive Tutorial: Building a Multi-Agent Economic Discussion Web Application with Crew AI, FastAPI, and Streamlit

Welcome to this comprehensive tutorial! In this guide, you'll learn how to build a web application where two intelligent agents discuss economic topics. We'll leverage the Crew AI starter template, integrate FastAPI for the backend, and use Streamlit for the frontend. By the end of this tutorial, you'll have a fully functional web app that allows users to input economic variables and view insightful discussions between the agents.

## Table of Contents

1. [Prerequisites](#prerequisites)
2. [Step 1: Set Up the Development Environment](#step-1-set-up-the-development-environment)
3. [Step 2: Modify the Crew AI Starter Template](#step-2-modify-the-crew-ai-starter-template)
   - [2.1. Define Custom Agents (`agents.py`)](#21-define-custom-agents-agentspy)
   - [2.2. Define Custom Tasks (`tasks.py`)](#22-define-custom-tasks-taskspy)
4. [Step 3: Create the FastAPI Backend](#step-3-create-the-fastapi-backend)
   - [3.1. Create `app.py`](#31-create-apppy)
   - [3.2. Running the FastAPI Server](#32-running-the-fastapi-server)
5. [Step 4: Create the Streamlit Frontend](#step-4-create-the-streamlit-frontend)
   - [4.1. Create `streamlit_app.py`](#41-create-streamlit_apppy)
   - [4.2. Running the Streamlit App](#42-running-the-streamlit-app)
6. [Step 5: Running the Complete Application](#step-5-running-the-complete-application)
7. [Step 6: Enhancements and Customizations](#step-6-enhancements-and-customizations)
   - [6.1. Adding More Agents](#61-adding-more-agents)
   - [6.2. Adding More Tasks](#62-adding-more-tasks)
   - [6.3. Integrating Additional Tools](#63-integrating-additional-tools)
   - [6.4. Improving the Frontend UI](#64-improving-the-frontend-ui)
8. [Summary](#summary)

---

## Prerequisites

Before we begin, ensure you have the following installed on your system:

- **Python 3.7 or higher**: [Download Python](https://www.python.org/downloads/)
- **Pip package manager**: Comes bundled with Python.
- **Git** (optional, for version control): [Download Git](https://git-scm.com/downloads)
- **Basic knowledge of Python programming**
- **Familiarity with FastAPI and Streamlit** (helpful but not required)

---

## Step 1: Set Up the Development Environment

### 1.1. Create a Project Directory

Start by creating a new directory for your project and navigating into it:

```sh
mkdir multi_agent_economic_discussion
cd multi_agent_economic_discussion
```

### 1.2. Set Up a Virtual Environment

It's good practice to use a virtual environment to manage your project's dependencies.

```sh
python3 -m venv venv
```

Activate the virtual environment:

- **On Windows:**

  ```sh
  venv\Scripts\activate
  ```

- **On macOS and Linux:**

  ```sh
  source venv/bin/activate
  ```

### 1.3. Install Required Dependencies

Install all necessary Python packages using `pip`:

```sh
python3 -m pip install --upgrade pip
pip install crewai langchain_openai duckduckgo-search langchain_ollama langchain_community fastapi uvicorn streamlit python-decouple
```

*Note*: Ensure you have the latest version of `duckduckgo-search` to avoid compatibility issues.

### 1.4. Set Up API Keys and Environment Variables

Create a `.env` file in the root of your project directory to store your API credentials securely.

```sh
touch .env
```

To open the .env file with Vim, you can use the following command in your terminal:

```sh
vim .env
```

Or open the `.env` file in your preferred text editor and add the following:

```env
OPENAI_API_KEY=<your_openai_api_key>
OPENAI_ORGANIZATION_ID=<your_openai_api_key>
```

Replace `<your_openai_api_key>` and `<your_openai_organization_id>` with your actual OpenAI credentials.

**Note**: If you're using Ollama, ensure it's installed and running on your local machine. Follow the [Ollama installation guide](https://ollama.com/) for setup instructions.

---

## Step 2: Modify the Crew AI Starter Template

We'll customize the provided Crew AI starter template by defining custom agents and tasks tailored to economic discussions.

### 2.1. Define Custom Agents (`agents.py`)

Create or modify the `agents.py` file to define the roles, backstories, and goals of your agents.

```sh
touch agents.py
```

Open `agents.py` and add the following code:

```python
# agents.py
from crewai import Agent, LLM
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_community.llms import Ollama

# This is an example of how to define custom agents.
# You can define as many agents as you want.
# You can also define custom tasks in tasks.py
class CustomAgents:
    def __init__(self):
        self.OpenAIGPT35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
        self.OpenAIGPT4 = ChatOpenAI(model_name="gpt-4", temperature=0.7)
        self.Ollama = LLM(model="ollama/llama3.2", base_url="http://localhost:11434")  # Specify the provider here

    def agent_1_name(self):
        return Agent(
            role="Economist specializing in fiscal policy",
            backstory="Experienced economist with expertise in analyzing government spending and taxation policies.",
            goal="To discuss and analyze the impact of fiscal policy on the economy.",
            llm=self.Ollama,  # Using Ollama for diverse insights
        )

    def agent_2_name(self):
        return Agent(
            role="Financial analyst focusing on monetary policy",
            backstory="Specializes in understanding central bank actions and their impact on inflation and economic stability.",
            goal="To evaluate the effect of monetary policy decisions on the economy.",
            llm=self.Ollama,  # Using Ollama for consistency
        )
```

#### Explanation:

- **Imports**:
  - `Agent` and `LLM` from `crewai` are used to define agents and their language models.
  - `ChatOpenAI` and `OllamaLLM` are language models from `langchain`.
  
- **CustomAgents Class**:
  - Initializes different language models (`gpt-3.5-turbo`, `gpt-4`, and `Ollama`).
  - Defines two agents:
    - **Agent 1**: An economist specializing in fiscal policy.
    - **Agent 2**: A financial analyst focusing on monetary policy.
  - Both agents use the `Ollama` language model for their responses.

### 2.2. Define Custom Tasks (`tasks.py`)

Create or modify the `tasks.py` file to define the tasks that the agents will perform.

```sh
touch tasks.py
```

Open `tasks.py` and add the following code:

```python
# tasks.py
from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __tip_section(self):
        return "If you do your BEST WORK, I'll give you a $10,000 commission!"

    def task_1_name(self, agent, var1, var2):
        return Task(
            description=dedent(
                f"""
                Provide an overview of the current fiscal policy's impact on the economy, considering variables like {var1} and {var2}.

                {self.__tip_section()}

                Make sure to use the most recent data to provide an accurate analysis.
            """
            ),
            expected_output="An analysis of fiscal policy's effect on economic growth and unemployment.",
            agent=agent,
        )

    def task_2_name(self, agent):
        return Task(
            description=dedent(
                f"""
                Analyze the impact of recent monetary policy decisions in response to the fiscal policy analysis provided by the first agent.

                {self.__tip_section()}
            """
            ),
            expected_output="A contrasting viewpoint on how monetary policy influences inflation and economic stability.",
            agent=agent,
        )
```

#### Explanation:

- **CustomTasks Class**:
  - **`__tip_section` Method**: Adds an incentive message to motivate the agents.
  - **`task_1_name` Method**: Assigns the first task to Agent 1, prompting an overview of fiscal policy's impact based on two variables (`var1` and `var2`).
  - **`task_2_name` Method**: Assigns the second task to Agent 2, prompting an analysis of monetary policy in response to Agent 1's analysis.

---

## Step 3: Create the FastAPI Backend

We'll set up a FastAPI backend that will handle requests from the frontend, run the multi-agent system, and return the results.

### 3.1. Create `app.py`

Create a new file named `app.py` in your project directory.

```sh
touch app.py
```

Open `app.py` and add the following code:

```python
# app.py
import os
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from decouple import config

from crewai import Crew
from agents import CustomAgents
from tasks import CustomTasks

from langchain.tools import DuckDuckGoSearchRun

# Initialize the DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()

# Load environment variables
os.environ["OPENAI_API_KEY"] = config("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = config("OPENAI_ORGANIZATION_ID")

# Define request and response schemas
class CrewRequest(BaseModel):
    var1: str
    var2: str

class CrewResponse(BaseModel):
    result: str

# Initialize FastAPI app
app = FastAPI(
    title="Crew AI API",
    description="API for running custom Crew AI tasks",
    version="1.0.0",
)

# Define the CustomCrew class
class CustomCrew:
    def __init__(self, var1: str, var2: str):
        self.var1 = var1
        self.var2 = var2

    def run(self) -> dict:
        agents = CustomAgents()
        tasks = CustomTasks()

        # Initialize custom agents
        agent1 = agents.agent_1_name()
        agent2 = agents.agent_2_name()

        # Initialize custom tasks
        task1 = tasks.task_1_name(agent1, self.var1, self.var2)
        task2 = tasks.task_2_name(agent2)

        # Create and run the crew
        crew = Crew(
            agents=[agent1, agent2],
            tasks=[task1, task2],
            verbose=True,
        )
        crew_output = crew.kickoff()

        return {"output": getattr(crew_output, 'raw', str(crew_output))}

# Define the API endpoint
@app.post("/run_agents", response_model=CrewResponse)
async def run_crew(crew_request: CrewRequest):
    """
    Run the custom Crew AI with provided variables.

    - **var1**: Economic variable 1 (e.g., inflation rate)
    - **var2**: Economic variable 2 (e.g., government spending)
    """
    try:
        custom_crew = CustomCrew(crew_request.var1, crew_request.var2)
        result = custom_crew.run()
        return CrewResponse(result=result["output"])
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Root endpoint for health check
@app.get("/")
def read_root():
    return {"message": "Welcome to the Crew AI API. Use /docs for API documentation."}

# Entry point for running the app
if __name__ == "__main__":
    uvicorn.run("your_module_name:app", host="127.0.0.1", port=8002, reload=True)
```

#### Explanation:

- **Imports**:
  - **FastAPI and Pydantic**: For creating API endpoints and data models.
  - **CrewAI Components**: `Crew` to manage agents and tasks.
  - **Environment Variables**: Loaded using `decouple`.
  - **CORS Middleware**: To allow cross-origin requests from the frontend.

- **CustomCrew Class**:
  - Initializes with two variables (`var1` and `var2`).
  - Instantiates agents and tasks.
  - Creates a `Crew` with the agents and tasks.
  - Runs the crew and returns the result.

- **API Endpoint**:
  - **`/run_agents`**: A POST endpoint that accepts `var1` and `var2` as input and returns the agents' discussion output.

### 3.2. Running the FastAPI Server

Start the FastAPI server using Uvicorn:

```sh
uvicorn app:app --reload
```

- **`app:app`**: Refers to the `app` object in `app.py`.
- **`--reload`**: Enables auto-reloading on code changes.

You should see output similar to:

```
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [28764] using statreload
INFO:     Started server process [28766]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

Your FastAPI backend is now running at `http://127.0.0.1:8000`.

---

## Step 4: Create the Streamlit Frontend

We'll build a Streamlit frontend that allows users to input economic variables and view the agents' discussions.

### 4.1. Create `streamlit_app.py`

Create a new file named `streamlit_app.py` in your project directory.

```sh
touch streamlit_app.py
```

Open `streamlit_app.py` and add the following code:

```python
# streamlit_app.py
import streamlit as st
import requests

# Streamlit app title
st.title("Multi-Agent Economic Discussion App")

# Instructions
st.markdown("""
Enter two economic variables to initiate the discussion between the Economist and Financial Analyst.
""")

# Input fields
var1 = st.text_input("Enter economic variable 1 (e.g., inflation rate):")
var2 = st.text_input("Enter economic variable 2 (e.g., government spending):")

# Button to run the agents
if st.button("Run Discussion"):
    if var1 and var2:
        with st.spinner("Running agents..."):
            try:
                # Call the FastAPI backend
                response = requests.post(
                    "http://127.0.0.1:8002/run_agents",
                    json={"var1": var1, "var2": var2}
                )
                if response.status_code == 200:
                    result = response.json()
                    st.success("Agents have completed their discussion!")
                    st.text_area("Discussion Output", result, height=300)
                else:
                    st.error(f"Error running agents: {response.status_code} - {response.text}")
            except requests.exceptions.RequestException as e:
                st.error(f"Connection error: {e}")
    else:
        st.warning("Please enter both variables.")
```

#### Explanation:

- **Imports**:
  - `streamlit`: For creating the frontend interface.
  - `requests`: For making HTTP requests to the FastAPI backend.

- **User Interface**:
  - **Title and Instructions**: Guides the user on what to do.
  - **Input Fields**: Two text inputs for economic variables.
  - **Run Discussion Button**: Triggers the agents' discussion.

- **Interaction Logic**:
  - When the button is pressed and both variables are provided:
    - Sends a POST request to the FastAPI backend with the input variables.
    - Displays a spinner while waiting for the response.
    - Upon successful response, displays the agents' discussion in a text area.
    - Handles errors gracefully, providing feedback to the user.

### 4.2. Running the Streamlit App

In a new terminal window (since the FastAPI server is running in another), activate your virtual environment if it's not already active, navigate to your project directory, and run:

```sh
streamlit run streamlit_app.py
```

You should see output similar to:

```
  You can now view your Streamlit app in your browser.

  Network URL: http://192.168.1.10:8501
  External URL: http://<your-ip>:8501
```

Open the provided URL (`http://localhost:8501`) in your web browser to access the Streamlit frontend.

---

## Step 5: Running the Complete Application

Now that both the FastAPI backend and Streamlit frontend are set up, you can run the complete application.

### 5.1. Ensure Both Servers Are Running

- **FastAPI Backend**: Should be running on `http://127.0.0.1:8000`.
- **Streamlit Frontend**: Should be running on `http://localhost:8501`.

### 5.2. Using the Application

1. **Open the Streamlit App**:
   
   Navigate to `http://localhost:8501` in your web browser.

2. **Enter Economic Variables**:
   
   - **Variable 1**: For example, `inflation rate`.
   - **Variable 2**: For example, `government spending`.

3. **Run the Discussion**:
   
   Click on the "Run Discussion" button. A spinner will appear indicating that the agents are processing.

4. **View the Results**:
   
   Once processing is complete, the agents' discussion output will appear in the text area below the button.

### 5.3. Example Output

```
Agent 1 (Economist specializing in fiscal policy):
The current fiscal policy, characterized by increased government spending and adjusted taxation rates, has a significant impact on the economy. By injecting more funds into public projects, the government stimulates economic growth, leading to higher employment rates and increased consumer spending. However, this can also result in higher inflation rates if the increased demand outpaces supply. Balancing these variables is crucial to maintain economic stability.

Agent 2 (Financial analyst focusing on monetary policy):
In response to the fiscal policy changes, the central bank may adjust monetary policies to control inflation and ensure economic stability. For instance, if government spending leads to higher inflation, the central bank might increase interest rates to curb excessive demand. Conversely, if the economy shows signs of slowing down despite increased spending, the central bank might lower interest rates to encourage borrowing and investment. This interplay between fiscal and monetary policies is essential for sustainable economic growth.
```

*Note*: The actual output may vary based on the models' responses and the current economic data available to them.

---

## Step 6: Enhancements and Customizations

To make your application more robust and feature-rich, consider implementing the following enhancements.

### 6.1. Adding More Agents

Introduce additional agents with different expertise to broaden the scope of discussions. For example:

- **Trade Analyst**: Focuses on international trade policies and their economic impact.
- **Economic Historian**: Provides historical context to current economic policies.
- **Market Strategist**: Analyzes market trends and investment strategies.

**Example: Adding a Trade Analyst**

1. **Update `agents.py`**:

   ```python
   def agent_3_name(self):
       return Agent(
           role="Trade analyst specializing in international trade policies",
           backstory="Expert in analyzing the effects of trade agreements and tariffs on global and domestic markets.",
           goal="To evaluate how current trade policies influence economic growth and international relations.",
           llm=self.Ollama,
       )
   ```

2. **Update `tasks.py`**:

   ```python
   def task_3_name(self, agent):
       return Task(
           description=dedent(
               f"""
               Assess the impact of recent international trade policies in light of the fiscal and monetary analyses provided by the previous agents.

               {self.__tip_section()}
           """
           ),
           expected_output="An evaluation of how trade policies affect economic growth and international relations.",
           agent=agent,
       )
   ```

3. **Update `app.py`**:

   ```python
   # In CustomCrew.run()
   custom_agent_3 = agents.agent_3_name()
   custom_task_3 = tasks.task_3_name(
       custom_agent_3,
   )

   # Update Crew
   crew = Crew(
       agents=[custom_agent_1, custom_agent_2, custom_agent_3],
       tasks=[custom_task_1, custom_task_2, custom_task_3],
       verbose=True,
   )
   ```

4. **Update `streamlit_app.py`** (if needed to display additional outputs).

### 6.2. Adding More Tasks

Expand the conversation by introducing more tasks, such as:

- **Summary Task**: Have an agent summarize the key points discussed.
- **Prediction Task**: Ask agents to predict future economic trends based on current policies.
- **Recommendation Task**: Have agents provide recommendations for policymakers.

**Example: Adding a Summary Task**

1. **Update `tasks.py`**:

   ```python
   def task_4_name(self, agent):
       return Task(
           description=dedent(
               f"""
               Summarize the key points discussed by the previous agents regarding fiscal and monetary policies.

               {self.__tip_section()}
           """
           ),
           expected_output="A concise summary of the discussions on fiscal and monetary policies.",
           agent=agent,
       )
   ```

2. **Update `app.py`**:

   ```python
   # Define a summarizer agent
   custom_agent_4 = agents.agent_4_name()  # Define this agent in agents.py

   # Define the summary task
   custom_task_4 = tasks.task_4_name(
       custom_agent_4,
   )

   # Update Crew
   crew = Crew(
       agents=[custom_agent_1, custom_agent_2, custom_agent_4],
       tasks=[custom_task_1, custom_task_2, custom_task_4],
       verbose=True,
   )
   ```

### 6.3. Integrating Additional Tools

Enhance agents' capabilities by integrating additional tools:

- **Web Search Tools**: Allow agents to fetch the latest data or news articles to inform their analysis.
- **Data Analysis Tools**: Enable agents to process and visualize economic data for more in-depth insights.

**Example: Integrating DuckDuckGo Search**

1. **Update `app.py`**:

   ```python
   from langchain.tools import DuckDuckGoSearchRun

   search_tool = DuckDuckGoSearchRun()

   # Pass tools to agents if supported
   custom_agent_1 = agents.agent_1_name(tools=[search_tool])
   custom_agent_2 = agents.agent_2_name(tools=[search_tool])
   ```

2. **Update Agent Definitions in `agents.py`** to accept tools.

### 6.4. Improving the Frontend UI

Enhance the user interface for better usability and aesthetics:

- **Streamlit Widgets**: Use dropdowns, sliders, or radio buttons for variable inputs.
- **Formatting**: Apply markdown or other formatting to improve readability.
- **Responsive Design**: Ensure the app looks good on different screen sizes.

**Example: Using Dropdowns for Variables**

```python
# streamlit_app.py

# Define options for variables
variable_options = {
    "Variable 1": ["Inflation Rate", "Unemployment Rate", "GDP Growth"],
    "Variable 2": ["Government Spending", "Taxation", "Interest Rates"]
}

var1 = st.selectbox("Select economic variable 1:", variable_options["Variable 1"])
var2 = st.selectbox("Select economic variable 2:", variable_options["Variable 2"])
```

---

## Summary

In this comprehensive tutorial, you learned how to build a multi-agent economic discussion web application using the Crew AI framework, FastAPI, and Streamlit. Here's a recap of what we covered:

1. **Environment Setup**: Created a project directory, set up a virtual environment, and installed necessary dependencies.
2. **Crew AI Customization**:
   - **Agents**: Defined custom agents with specific roles, backstories, and goals using both OpenAI and Ollama language models.
   - **Tasks**: Created tasks that guide the agents' discussions based on user-provided economic variables.
3. **FastAPI Backend**: Developed a FastAPI backend that handles incoming requests, runs the multi-agent system, and returns the discussion output.
4. **Streamlit Frontend**: Built a user-friendly frontend with Streamlit that allows users to input variables and view the agents' discussions in real-time.
5. **Running the Application**: Successfully ran both backend and frontend servers to operate the complete application.
6. **Enhancements and Customizations**: Explored ways to expand the application's capabilities by adding more agents, tasks, integrating additional tools, and improving the frontend UI.

By following this tutorial, you've created a powerful tool that simulates expert discussions on economic topics, providing valuable insights and fostering a deeper understanding of complex economic policies.

### Next Steps

- **Deployment**: Consider deploying your application to a cloud platform for broader access. Services like **Heroku**, **AWS**, **Google Cloud**, or **Streamlit Sharing** are excellent options.
- **Scalability**: Implement features to handle multiple concurrent users and large-scale discussions.
- **Security**: Secure your API endpoints and manage sensitive data responsibly.
- **User Feedback**: Incorporate user feedback mechanisms to continuously improve the application's functionality and user experience.

Feel free to experiment with different configurations, add more sophisticated agents, and tailor the discussions to various economic scenarios or other domains of interest. Happy coding!

---

